# Open Street Map Analysis
### Course: Data Wrangling with MongoDB

Content: Processing of OSM Dataset
- Audit OSM XML
- Clean Data
- Convert XML Data to JSON Format
- Store Data in mongoDB
- Investigate Data in mongoDB


Note: The Report about this project is stored in a separated file (Data_Analyst_ND_Project_3_ProjectReport.html).

## Process OSM Dataset

Thoroughly audit and clean your dataset, converting it from XML to JSON format. It is recommended that you start with the Lesson 6 exercises and modify them to suit your chosen data set. As you unravel the data, take note of problems encountered along the way as well as issues with the dataset. You are going to need these when you write your project report. Finally, import the clean JSON file into a MongoDB database and run some queries against it.

### Import Libraries

Include all the Libaries which will be used later on in Python code.

In [1]:
import xml.etree.cElementTree as ET
import pandas as pd
import pprint
import re
import codecs
import xmltodict
import json
import itertools
import pymongo
import os
import operator
import string

from collections import defaultdict

### File

Set filename of file being analysed. File is located in the folder of this iPython project / Jupyter notebook.

In [2]:
filename = 'waukesha_county.osm'

### Regular Expressions

Setting up some regular expression patterns which will be used later on.

In [3]:
# patterns to find different types of tags
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# street name pattern to find abbreviations
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# post code pattern
post_code_re = re.compile(r'53[0-5][0-9][0-9]')

Setting up variable for later usage during data analysis and cleaning. This includes the mapping information for street names as well as a list of all post codes for Waukesha County which is the subject for investigation.

In [4]:
# Tag categories
keys_tag = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}

# Mapping of shortcuts for cleaning of street name values
city_name_correction_mapping = { 
    "MILWAUKEE" : "Milwaukee",    
    "MIlwaukee" : "Milwaukee", 
    "milwaukee" : "Milwaukee",
    "Waukesa"   : "Waukesha", 
    "waukesha"   : "Waukesha"}

# Address/Street 
street_name_expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
                        "Trail", "Parkway", "Commons", "Highway", "Way"]

# Mapping of shortcuts for cleaning of street name values
street_name_mapping = {              "St"      : "Street",    "St."     : "Street",     "Rd"      : "Road",     
            "Rd."     : "Road",      "Ave"     : "Avenue",    "Ave."    : "Avenue",     "Bulvard" : "Boulevard",  
            "Blvd."   : "Boulevard", "Blvd"    : "Boulevard", "Dr"      : "Drive",      "Dr."     : "Drive",      
            "Ct"      : "Court",     "Ct."     : "Court",     "Crt"     : "Court",      "Crt."    : "Court",  
            "Pl."     : "Place",     "PL"      : "Place",     "Cir"     : "Circus",     "Tr."     : "Track",
            "Plc"     : "Place",     "Plc."    : "Place",     "Sqr"     : "Square",     "Sqr."    : "Square",     
            "Squ"     : "Square",    "Squ."    : "Square",    "Sq"      : "Square",     "Sq."     : "Square",     
            "Ln"      : "Lane",      "Ln."     : "Lane",      "Trl"     : "Trail",      "Trl."    : "Trail",      
            "Pky"     : "Parkway",   "Pky."    : "Parkway",   "Pkwy"    : "Parkway",    "Pkwy."   : "Parkway",    
            "Hwy"     : "Highway",   "Hwy."    : "Highway",   "R."      : "Road",       "W."      : "Way",
            "Wy."     : "Way",       "Wy"      : "Way",       "Comm."   : "Commons",    "Cms."    : "Commons",
            "N"       : "North",     "E"       : "East",      "S"       : "South",      "W"       : "West"
            }

# Expteced post codes in Waukesha County - list is retrieved from: 
# http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=wi&StName=wisconsin&StFIPS=&FIPS=55133
post_code_expected = ["53005", "53007", "53008", "53018", "53029", "53045", "53046", "53051", "53052", "53056", "53058",
                      "53064", "53066", "53069", "53072", "53089", "53103", "53118", "53119", "53122", "53127", "53146", 
                      "53149", "53150", "53151", "53153", "53183", "53186", "53187", "53188", "53189"] 

### Functions for Data Analysis

In [5]:
# Count Different Tags in the Data 
# is retrieving all different kinds of tags used within the data. A very basic function to get some informatio about the data.
# INPUT: filename - name of XML file
# RETURN: data - dict with different tags and related occurence in the data
def count_tags(filename):
    # dict for tags
    data = dict()
    for event, elem in ET.iterparse(filename):
        if elem.tag in data.keys():
            noo = data[elem.tag]
            data.update({elem.tag:noo+1})
        else:
            data.update({elem.tag:1})
    return data

In [6]:
# Count Different Types of Tags in the Data
# Different types are defined by certain regex definitions earlier in the file
# INPUT: element - element in XML data
# INPUT: keys - dict to store the occurence of specific types
# RETURN: keys
def key_type(element, keys):
    for attributes in element.findall("tag"):
        nameK = attributes.get("k")
        if lower.match(nameK):
            keys["lower"] = keys["lower"]+1
        elif lower_colon.match(nameK):
            keys["lower_colon"] = keys["lower_colon"]+1
        elif problemchars.match(nameK):
            keys["problemchars"] = keys["problemchars"]+1
        else:
            keys["other"] = keys["other"]+1
    return keys

In [7]:
# List of unique users in the data
# INPUT: filename - name of XML file
# RETURN: users - dict with user list and number of contributions
def unique_users(filename):
    users = dict()
    for event, elem in ET.iterparse(filename):
        user = elem.get('user')
        if user != None:
            if user in users.keys():
                noo = users[user]
                users.update({user:noo+1})
            else:
                users.update({user:1})
    return users

In [8]:
########################################################################
# check if a tag is a certain type relevant for specific actions
########################################################################

# Address data checks
##############################################################
# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_housenumber(elem):
    return (elem.attrib['k'] == "addr:housenumber")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_post_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_city(elem):
    return (elem.attrib['k'] == "addr:city")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_state(elem):
    return (elem.attrib['k'] == "addr:state")
##############################################################

# Generic Info data checks
##############################################################
# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_name(elem):
    return (elem.attrib['k'] == "name")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_amenity(elem):
    return (elem.attrib['k'] == "amenity")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_opening_hours(elem):
    return (elem.attrib['k'] == "opening_hours")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_phone(elem):
    return (elem.attrib['k'] == "phone")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_website(elem):
    return (elem.attrib['k'] == "website")

# Cusisine data checks
##########################################
# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_cuisine(elem):
    return (elem.attrib['k'] == "cuisine")

# Relegion Info data checks
##########################################
# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_denomination(elem):
    return (elem.attrib['k'] == "denomination")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_religion(elem):
    return (elem.attrib['k'] == "religion")

# Shop Info data checks
##########################################
# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_shop(elem):
    return (elem.attrib['k'] == "shop")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_highway(elem):
    return (elem.attrib['k'] == "highway")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_building(elem):
    return (elem.attrib['k'] == "building")

# INPUT: element - element in XML data
# RETURN: boolean - true if tag fits condition 
def is_type(elem):
    return (elem.attrib['k'] == "type")

##############################################################

In [9]:
# This creates a set of street names for different types of streets where the name does not fit to the 
# expected street name pattern. Thus the function helps to identify streetname of bad quality.
# INPUT: type_set - set of different streets
# INPUT: value - streetname
def audit_street_type(type_set, value):
    naming = street_type_re.search(value)
    if naming:
        stype = naming.group()
        found = False
        for k in street_name_mapping.keys():
            if ((k in value) or (street_name_mapping[k] in value) or (stype in street_name_expected)):
                found = True
        if not found:
            type_set[stype].add(value)    

In [10]:
# this creates a set of post codes for different where the post code does not fit to the 
# expected post code pattern of Waukesha County. Thus the function helps to identify wrong labled
# post codes in the data.
# INPUT: type_set - set of different streets
# INPUT: value - streetname
def audit_post_code_types(type_set, value):
    if value not in post_code_expected:
        if value in type_set.keys():
            noo = type_set[value]
            type_set[value] = noo+1
        else:
            type_set[value] = 1

In [11]:
# audit amenity type function collects all different types of amenity and their occurences in the data
# INPUT: type_set - set of amenities
# INPUT: value - amenity
def audit_type(type_set, value):
    if value in type_set.keys():
        noo = type_set[value]
        type_set[value] = noo+1
    else:
        type_set[value] = 1

In [12]:
# main function to audit the data
# INPUT: osmfile - original XML file from OSM
# RETURN: tag_types - dict with different types of tags and occurences
# RETURN: street_types  - dict with different streetnames and occurences
# RETURN: amenity_types  - dict with different amenities and occurences
# RETURN: post_code_types - dict with different post codes and occurences
# RETURN: wrong_post_code_types - dict with wrong post codes failing validation and occurences
# RETURN: cities_dict - dict with different cities and occurences
def audit(osmfile):
    
    # open file
    osm_file = open(osmfile, "r")
    
    # container for analysis
    street_types= defaultdict(set) # container for unexpected street name occurences
    amenity_types = dict() # container for all different types of amenity and related number of occurence
    cities_dict = dict() # container for all different cities and related number of occurence
    post_code_types = dict() # container for all different types of amenity and related number of occurence
    wrong_post_code_types = dict() # container for all different types of wrong post codes and related number of occurence
    tag_types = dict() # container for different types of tags and related number of occurence
    
    # loop data
    for event, elem in ET.iterparse(osm_file):
        
        ##########################################################
        # START: Counting tag types
        if elem.tag in tag_types.keys():
            noo = tag_types[elem.tag]
            tag_types.update({elem.tag:(noo+1)})
        else:
            tag_types.update({elem.tag:1})
        # END
        ##########################################################
        
        # looking into node, way and realtions tags only
        if elem.tag in ("node", "way", "relation"):
            
            # iterating through tag elements in node/way
            for tag in elem.iter("tag"):
                
                ##########################################################
                # START: analysing street name tags
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                # END
                ##########################################################
                
                ##########################################################
                # START: analysing amenity tags
                if is_amenity(tag):
                    audit_type(amenity_types, tag.attrib['v'])
                # END
                ##########################################################
                
                ##########################################################
                # START: analysing post code tags
                if is_post_code(tag):
                    # list of all different post codes in the data and their occurence
                    audit_type(post_code_types, tag.attrib['v'])
                    # audit if all the post codes are related to waukesha county area
                    audit_post_code_types(wrong_post_code_types, tag.attrib['v'])
                # END
                ##########################################################
                
                ##########################################################
                # START: analysing post code tags
                if is_city(tag):
                    # list of all different post codes in the data and their occurence
                    audit_type(cities_dict, tag.attrib['v'])
                # END
                ##########################################################
                    
    osm_file.close()
    
    return tag_types, street_types, amenity_types, post_code_types, wrong_post_code_types, cities_dict

### Data Analysis

In [13]:
# Return the size, in bytes, of path. Raise os.error if the file does not exist or is inaccessible.
print "Size of OSM data file: "
print (str(os.path.getsize(filename)/1024)) + " KB"

Size of OSM data file: 
192404 KB


#### Different Key Types and Number of Occurence

In [17]:
# analysis of the different keys (k) used inside the tag element
for id, element in ET.iterparse(filename):
    keys_tag = key_type(element, keys_tag)
################################################
pprint.pprint("--------------- Key Types & Count ------------------")
pprint.pprint("Different Key Types and Number of Occurence:")
pprint.pprint("----------------------------------------------------")
pprint.pprint(keys_tag)

# Need to free up variables
# in order to get RAM utilization managed#
element = 0

'--------------- Key Types & Count ------------------'
'Different Key Types and Number of Occurence:'
'----------------------------------------------------'
{'lower': 718689, 'lower_colon': 804417, 'other': 35025, 'problemchars': 0}


The function is using regex definition from earlier in this file. What's of most interest here is, that no tags with problematic characters are in the file.

#### List of Unique Users

In [19]:
# list of unique users contributing to osm data
users = unique_users(filename)
################################################
pprint.pprint("--------------- Users ------------------")
pprint.pprint("Number of unique users: " + str(len(users)))
pprint.pprint("----------------------------------------") 
pprint.pprint("Top 10 users in the list and the number")
pprint.pprint("of posts:")
pprint.pprint("----------------------------------------") 
for i in range(0, 10):
    pprint.pprint(sorted(users.items(), key=operator.itemgetter(1), reverse=True)[i])

# Need to free up variables
# in order to get RAM utilization managed
users = None

'--------------- Users ------------------'
'Number of unique users: 682'
'----------------------------------------'
'Top 10 users in the list and the number'
'of posts:'
'----------------------------------------'
('woodpeck_fixbot', 217089)
('ItalianMustache', 102449)
('shuui', 94917)
('hogrod', 48592)
('reschultzed', 40931)
('bbauter', 29704)
('Mulad', 28162)
('Gary Cox', 27193)
('iandees', 26923)
('TIGERcnl', 25893)


#### Various Insights and Numbers related to the Data

In [20]:
# calling audit function to collect auditing information about the data
tag_types, street_types, amenity_types, post_code_types, wrong_post_code_types, cities_dict = audit(filename)

In [21]:
pprint.pprint("------------------- Tag Types ----------------------")
pprint.pprint("Below is a list of different Tags and and their")
pprint.pprint("occurence in the data.")
pprint.pprint("----------------------------------------------------")
pprint.pprint(sorted(tag_types.items(), key=operator.itemgetter(1), reverse=True))
pprint.pprint("----------------------------------------------------")

'------------------- Tag Types ----------------------'
'Below is a list of different Tags and and their'
'occurence in the data.'
'----------------------------------------------------'
[('nd', 1052051),
 ('node', 860835),
 ('tag', 519377),
 ('way', 92239),
 ('member', 17750),
 ('relation', 786),
 ('note', 1),
 ('meta', 1),
 ('bounds', 1),
 ('osm', 1)]
'----------------------------------------------------'


In [22]:
pprint.pprint("------------------ Street Types --------------------")
pprint.pprint("Below street names are those which do not fit into ")
pprint.pprint("street names or street names abbrivations which are")
pprint.pprint("defined earlier in street_name_mapping variable.")
pprint.pprint("----------------------------------------------------")
for st_t in street_types:
    pprint.pprint(st_t + ":")
    pprint.pprint(street_types[st_t])
    pprint.pprint("-------------------------------")
pprint.pprint("----------------------------------------------------")

'------------------ Street Types --------------------'
'Below street names are those which do not fit into '
'street names or street names abbrivations which are'
'defined earlier in street_name_mapping variable.'
'----------------------------------------------------'
'53076:'
set(['53076'])
'-------------------------------'
u'napaonline.com\u200e:'
set([u'napaonline.com\u200e'])
'-------------------------------'
'Pointe:'
set(['Gaslight Pointe'])
'-------------------------------'
'----------------------------------------------------'


In [23]:
pprint.pprint("------------------ Amenity Types -------------------")
pprint.pprint("Below is a list of different amenities and and their")
pprint.pprint("occurence in the data.")
pprint.pprint("----------------------------------------------------")
for i in range(0, 10):
    pprint.pprint(sorted(amenity_types.items(), key=operator.itemgetter(1), reverse=True)[i])
pprint.pprint("----------------------------------------------------")
pprint.pprint("")

'------------------ Amenity Types -------------------'
'Below is a list of different amenities and and their'
'occurence in the data.'
'----------------------------------------------------'
('parking', 2305)
('school', 997)
('parking_entrance', 569)
('restaurant', 444)
('fast_food', 230)
('grave_yard', 199)
('fuel', 192)
('place_of_worship', 171)
('bench', 142)
('bank', 110)
'----------------------------------------------------'
''


In [24]:
pprint.pprint("----------------- Post Codes ------------------")
pprint.pprint("Below is a list of all post codes and and their")
pprint.pprint("occurence in the data.")
pprint.pprint("----------------------------------------------------")
pprint.pprint("Number of different Post Codes: " + str(len(post_code_types)))
pprint.pprint("----------------------------------------------------")
for i in range(0, 10):
    pprint.pprint(sorted(post_code_types.items(), key=operator.itemgetter(1), reverse=True)[i])
pprint.pprint("----------------------------------------------------")
pprint.pprint("")

'----------------- Post Codes ------------------'
'Below is a list of all post codes and and their'
'occurence in the data.'
'----------------------------------------------------'
'Number of different Post Codes: 98'
'----------------------------------------------------'
('53202', 993)
('53212', 255)
('53203', 229)
('53538', 143)
('53233', 132)
('53211', 93)
('53204', 67)
('53215', 47)
('53094', 42)
('53027', 38)
'----------------------------------------------------'
''


In [25]:
pprint.pprint("--------------- Correct Post Codes -----------------")
pprint.pprint("Below is a list of post codes which do belong to")
pprint.pprint("Waukesha County and their occurence in the data.")
pprint.pprint("----------------------------------------------------")
sorted_post_codes = sorted(post_code_types.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint("Number of different Correct Post Codes: " + str((len(sorted_post_codes)-(len(wrong_post_code_types)))))
pprint.pprint("----------------------------------------------------")
for post_code in sorted_post_codes:
    if post_code[0] not in wrong_post_code_types.keys():
        print post_code
pprint.pprint("----------------------------------------------------")
pprint.pprint("")

'--------------- Correct Post Codes -----------------'
'Below is a list of post codes which do belong to'
'Waukesha County and their occurence in the data.'
'----------------------------------------------------'
'Number of different Correct Post Codes: 22'
'----------------------------------------------------'
('53029', 23)
('53186', 23)
('53066', 12)
('53072', 11)
('53045', 11)
('53051', 10)
('53005', 9)
('53188', 8)
('53151', 7)
('53146', 3)
('53149', 3)
('53069', 3)
('53089', 2)
('53150', 2)
('53122', 2)
('53189', 2)
('53153', 1)
('53007', 1)
('53018', 1)
('53183', 1)
('53187', 1)
('53103', 1)
'----------------------------------------------------'
''


In [26]:
pprint.pprint("----------------- Wrong Post Codes -------------------")
pprint.pprint("Below is a list of post codes which do not belong to")
pprint.pprint("Waukesha County and their occurence in the data.")
pprint.pprint("------------------------------------------------------")
pprint.pprint("Number of different Post Codes: " + str(len(wrong_post_code_types)))
pprint.pprint('CAUTION: This is a list of the TOP 10 Wrong Post Codes')
pprint.pprint("------------------------------------------------------")
for i in range(0, 10):
    pprint.pprint(sorted(wrong_post_code_types.items(), key=operator.itemgetter(1), reverse=True)[i])
pprint.pprint("------------------------------------------------------")
pprint.pprint("")

'----------------- Wrong Post Codes -------------------'
'Below is a list of post codes which do not belong to'
'Waukesha County and their occurence in the data.'
'------------------------------------------------------'
'Number of different Post Codes: 76'
'CAUTION: This is a list of the TOP 10 Wrong Post Codes'
'------------------------------------------------------'
('53202', 993)
('53212', 255)
('53203', 229)
('53538', 143)
('53233', 132)
('53211', 93)
('53204', 67)
('53215', 47)
('53094', 42)
('53027', 38)
'------------------------------------------------------'
''


In [27]:
pprint.pprint("---------------- Wrong Post Codes ------------------")
pprint.pprint("Below is a list of post codes which do not match an")
pprint.pprint("extended pattern around Waukesha County and their")
pprint.pprint("occurence in the data.")
pprint.pprint("Pattern: (1) 53000 - 53599 and")
pprint.pprint("         (2) length of post code is 5 chars")
pprint.pprint("----------------------------------------------------")
for post_code in wrong_post_code_types:
    # check agains regex for valid post codes
    is_valid_post_code = post_code_re.match(post_code)
    # if post code does not match expected pattern or post code length is not 5
    if (not is_valid_post_code) or (len(post_code) != 5):
        pprint.pprint(post_code)
pprint.pprint("----------------------------------------------------")
pprint.pprint("")

'---------------- Wrong Post Codes ------------------'
'Below is a list of post codes which do not match an'
'extended pattern around Waukesha County and their'
'occurence in the data.'
'Pattern: (1) 53000 - 53599 and'
'         (2) length of post code is 5 chars'
'----------------------------------------------------'
'53403-9998'
'Milwaukee WI, 53222'
'54220'
'WI'
'53214-3110'
'53203-3099'
'1729'
'53217-5399'
'----------------------------------------------------'
''


In [28]:
pprint.pprint("---------------- Different Cities ------------------")
pprint.pprint("Below is a list of different cities mentioned in the")
pprint.pprint("data and their occurence.")
pprint.pprint("----------------------------------------------------")
for i in range(0, 10):
    pprint.pprint(sorted(cities_dict.items(), key=operator.itemgetter(1), reverse=True)[i])
pprint.pprint("----------------------------------------------------")
pprint.pprint("")

'---------------- Different Cities ------------------'
'Below is a list of different cities mentioned in the'
'data and their occurence.'
'----------------------------------------------------'
('Milwaukee', 1592)
('Racine', 561)
('Fort Atkinson', 145)
('Mount Pleasant', 120)
('Waterloo', 57)
('Watertown', 46)
('Waukesha', 31)
('Caledonia', 29)
('Sturtevant', 27)
('MIlwaukee', 23)
'----------------------------------------------------'
''


In [29]:
# Need to free up variables
# in order to get RAM utilization managed
tag_types = {}
street_types = {}
amenity_types = {}
post_code_types = {}
wrong_post_code_types = {}
cities_dict = {}

In [74]:
# Note: Below code was just to investigate a problem on the consistency of data classification between
# data in XML and data in JSON format.

# Get IDs of all different school nodes in the data set
diff_restaurant = set()

c=0

for event, elem in ET.iterparse(filename):

    # looking into node, way and realtions tags only
    if elem.tag in ("node", "way", "relation"):
        
        for tag in elem.iter("tag"):
            if (tag.attrib['k'] == "addr:postcode"):
                if tag.attrib['v'] == "53202":
                    c = c+1
                    diff_restaurant.add(elem.attrib.get("id"))
                    
print "Number of restaurant in Set():"
print len(diff_restaurant)
print "Number of iterations in \"school\" condition:"
print c

print "List all school id's:"
for rest in diff_restaurant:
    print rest

# Need to free up variables
# in order to get RAM utilization managed   
diff_restaurant = set()
event = 0
elem = 0
c=0

Number of restaurant in Set():
993
Number of iterations in "school" condition:
993
List all school id's:
392886197
392886196
1938373853
180987114
395662863
1157330189
184082104
369923548
184082102
392886199
183095129
53216812
394657139
53216818
3913173676
117040603
3882343685
51865486
827596583
68798474
5937219
90547171
183340708
53160857
298358438
53160858
3969864878
393262123
393262126
395178071
180987111
55262352
392886198
51865535
53216288
116895728
3983518158
3960987399
115013002
53160488
53160561
3969864882
93829843
3969864880
93829846
395662860
395662861
183171727
53160357
395662864
184082092
395662868
3986531514
183249500
183171728
3986531513
107937238
255437342
53160687
393233755
53160682
191237612
392848483
392848482
392848481
394657136
395889444
3971992781
394706133
394706131
394706136
394706138
394706139
381257038
181033244
181033247
394686801
3993030972
384574237
1318516485
1050724370
383142389
2611216473
394227005
101476839
393233753
3985506873
396092145
394409366
3995771

### Data Cleaning and Conversion to JSON

In this section the data file will be converted from XML to JSON format. At the same time some data cleaning activities will be performed. The resulting JSON data will be cleaned already.

The need for data cleaning of different aspects of the data was analyst in earlier sections. This section picks up that information and develops strategies to clean up the data.

#### Functions for Data Cleaning and Coversion

In [200]:
# Attribute containers
PRIMARY  = ["id"] # main info from node attributes
CREATION = ["uid", "user", "changeset", "version", "timestamp"] # info about creation of data point
GEOPOS   = ["lon", "lat"] # info about geo location
ADDRESS  = "addr:" # address info
BUILDING = "building:" # building information
CONTACT = "contact:"

# Attribute containers for NODES
N_PRIMARY_TAG = ["name", "opening_hours", "operator", "phone", "website", "source"] # info considered to be main info, but from inside TAG
N_DEMOINFO = ["place", "population"] # demographic information
N_ADDINFO  = ["amenity", "cuisine", "fuel:diesel", "fuel:e10", "fuel:octane_91", "fuel:octane_95", "fuel:octane_98", "brand", \
            "shop", "denomination", "religion", "highway"] # additional info about data point from inside TAG

# Attribute containers for WAYS
W_PRIMARY_TAG = ["name", "source", "operator", "phone", "website"]
W_OBJECT_TAG = ["highway", "leisure", "natural", "building"] # info considered to be main info, but from inside TAG
W_BUILDING_ATTS = ["architect", "name:historic"] # building information
W_ADDINFO  = ["oneway", "horse", "foot", "bicycle", "lanes", "oneway", "sidewalk", "amenity"] # additional info about data point from inside TAG

# Attribute containers for WAYS
R_PRIMARY_TAG = ["name", "source", "operator", "phone", "website"]
R_OBJECT_TAG = ["highway", "leisure", "natural", "building"] # info considered to be main info, but from inside TAG
R_BUILDING_ATTS = ["architect", "name:historic"] # building information
R_ADDINFO  = ["place", "boundary", "admin_level","oneway", "amenity"] # additional info about data point from inside TAG

# GNIS Attributes (USGS Geographic Names Information System) Pattern
GNIS = "gnis:"
'''
    - Detailled Info about the attributes: http://wiki.openstreetmap.org/wiki/USGS_GNIS
    - Sample Attributes:
        ele = elevation
        gnis:Class = Feature Class name
        gnis:County = County name
        gnis:County_num = County FIPS code
        gnis:ST_alpha = State name (2-Letter abbreviation)
        gnis:ST_num = State FIPS code
        gnis:county_id = County FIPS code
        gnis:created = MM/DD/YYYY when the GNIS entry was created
        gnis:state_id = State FIPS code
'''
    
# TIGER Attribute (Topologically Integrated Geographic Encoding and Referencing system) Pattern
TIGER = "tiger:"
''' 
    - Detailled Info about the attributes: http://wiki.openstreetmap.org/wiki/TIGER_to_OSM_Attribute_Map
    - Sample Attributes:
        tiger:cfcc
        tiger:county
        tiger:name_base
        tiger:name_direction_prefix
        tiger:name_type
        tiger:reviewed
        tiger:from_address_right
        tiger:to_address_right
        tiger:from_address_left
        tiger:to_address_left
        tiger:zip_left
        tiger:zip_right
        tiger:tlid
        tiger:tzid
'''

''' 
Above the main constant variable have been defined and can be used later on in the analysis.
'''

' \nAbove the main constant variable have been defined and can be used later on in the analysis.\n'

In [32]:
# update street names and update abbreviations with long term
# problem with this function is that sometimes the names 
# are getting update wrongly
# INPUT: name - street name
# INPUT: mapping - street names to street name abbreviations
def update_name_old(name, mapping):
    for expr in mapping.keys():
        if (expr in name) and (mapping[expr] not in name):
            return name.replace(expr, mapping[expr])
    return name

# update street names and update abbreviations with long term
# note: this function is based upon a solution provided at https://github.com/batmanbury/Udacity
# INPUT: name - street name
# INPUT: mapping - street names to street name abbreviations
# RETURN: updated street name
def update_name(name, mapping):
    # split words in name
    words = name.split()
    # look at each word
    for w in range(len(words)):
        # check if word is set up in mapping variable
        if words[w] in mapping:
            words[w] = mapping[words[w]]
    name = " ".join(words)
    return name

In [33]:
# remove state abbr. of city tags
# INPUT: name - street name
# RETURN: updated street name
def remove_state_abbr(name):
    if ", " in name:
        temp =  name.lower().replace(", wi", "")
        return string.capwords(temp)
    else:
        return string.capwords(name)

In [34]:
# corrects some wrongly spelled street names based on a dictionary
# INPUT: name - street name
# RETURN: name - updated street name
def correct_city_spelling(name):
    if name in city_name_correction_mapping.keys():
        return city_name_correction_mapping[name]
    else:
        return name

In [35]:
# removes "-" and "_" from strings
# INPUT: name - certain string
# RETURN: temp - updated string
def remove_chars(name):
    temp =  name.lower().replace("-", " ")
    temp =  name.lower().replace("_", " ")
    return temp

In [54]:
# convert a node object to a python dict / JSON object string
# INPUT: elem - element in XML data
# RETURN: node - in JSON string format
def convert_node_to_json(elem):
    
    # NODE dict
    node = {} # main information container
    
    # SUB dicts
    creation = {}  # basic info about data creation
    address  = {}  # geopraphic information
    demoinfo = {}  # demographic information
    geopos   = {}  # geopraphic information
    addinfo  = {}  # some additional info about the datapoint
    gnis     = {}  # all GNIS attributes for the datapoint
    tiger    = {}  # all TIGER attributes for the datapoint
    other    = {}  # all other info
    
    # type is always "node" 
    node["type"] = "node" 
    
    # main attributes
    for key in elem.attrib:
        
        # grapping the main attributes
         ##############################################
        # main node info
        if key in PRIMARY:
            node[key] = elem.attrib.get(key) 
        
        ##############################################
        # sub node info
        elif key in CREATION:
            creation[key] = elem.attrib.get(key) 
                
        elif key in GEOPOS:
            geopos[key] = elem.attrib.get(key)
    
    # grapping sub info
    for tag in elem.iter():
        if tag.tag == "tag":
            
            ##############################################
            # main node info
            if tag.attrib['k'] in N_PRIMARY_TAG:
                 node[tag.attrib['k']] = tag.attrib['v']
            
            ##############################################
            # sub node info
            ##### all atts for creation dict
            elif tag.attrib['k'] in CREATION:
                creation[tag.attrib['k']] = tag.attrib['v']  
             
            ##### all atts for demographic information dict    
            elif tag.attrib['k'] in N_DEMOINFO:
                demoinfo[tag.attrib['k']] = tag.attrib['v']  
                
            ##### all atts for address dict
            elif ADDRESS in tag.attrib['k'] :
                 # cleaning street data
                if is_street_name(tag):
                    address[tag.attrib['k'].replace("addr:", "")] = update_name(tag.attrib['v'], street_name_mapping)
                # cleaning city data
                elif is_city(tag):
                    address[tag.attrib['k'].replace("addr:", "")] = correct_city_spelling(remove_state_abbr(tag.attrib['v']))
                # all other cases
                else:
                    address[tag.attrib['k'].replace("addr:", "")] = tag.attrib['v']  
                
            ##### all atts for additional information dict
            elif tag.attrib['k'] in N_ADDINFO:
                
                # special case for fuel stations
                if is_amenity(tag):
                    if tag.attrib['v'] == "fuel":
                        addinfo["type"] = "shop"
                        addinfo["shop_type"] = "fuel"
                    else:
                        addinfo["type"] = remove_chars(tag.attrib['v'])
                
                # other types of shops
                elif is_shop(tag):
                    try:
                        if len(addinfo["type"]) == 0:
                            pass
                        else:
                            addinfo["type_details"] = "shop"
                            addinfo["shop_type"] = remove_chars(tag.attrib['v'])
                    except:
                        addinfo["type"] = "shop"
                        addinfo["shop_type"] = remove_chars(tag.attrib['v'])
                    
                # other types of shops
                elif is_highway(tag):
                    addinfo["street_info"] = tag.attrib['v']
                
                else:
                    addinfo[tag.attrib['k']] = tag.attrib['v']  
                
            ##### all atts for GNIS information dict
            elif GNIS in tag.attrib['k'] or tag.attrib['k'] == 'ele':
                gnis[tag.attrib['k'].replace("gnis:", "")] = tag.attrib['v']  
                
            ##### all atts for TIGER information dict
            elif TIGER in tag.attrib['k']:
                tiger[tag.attrib['k'].replace("tiger:", "")] = tag.attrib['v']  
            
            ##### all atts for other dict
            else:
                other[tag.attrib['k']] = tag.attrib['v']   
            
    
    # put all the pieces together as they are available
    if len(creation) > 0:
        node["creation"] = creation
    if len(address) > 0:
        node["address"]  = address
    if len(demoinfo) > 0:
        node["demoinfo"] = demoinfo
    if len(geopos) > 0:
        node["geopos"]   = geopos
    if len(addinfo) > 0:
        node["addinfo"]  = addinfo
    if len(gnis) > 0:
        node["gnis"]     = gnis
    if len(tiger) > 0:
        node["tiger"]    = tiger
    if len(other) > 0:
        node["other"]    = other
        
    # Need to free up variables
    # in order to get RAM utilization managed
    creation = {}  
    address  = {}  
    demoinfo = {}  
    geopos   = {}  
    addinfo  = {}  
    gnis     = {}  
    tiger    = {}  
    other    = {}  
        
    # return the node as a dict object
    return node
    

In [53]:
# convert a way object to a python dict / JSON object string
# INPUT: elem - element in XML data
# RETURN: node - in JSON string format
def convert_way_to_json(elem):
    # NODE dict
    way = {} # main information container
    
    # SUB dicts
    creation = {}  # basic info about data creation
    address  = {}  # geopraphic information
    contact  = {}  # contact information
    building = {}  # building information
    area     = {}  # area information
    geopos   = {}  # geopraphic information
    addinfo  = {}  # some additional info about the datapoint
    gnis     = {}  # all GNIS attributes for the datapoint
    tiger    = {}  # all TIGER attributes for the datapoint
    nds      = []   # nd references
    other    = {}  # all other info
    
    # type is always "node" 
    way["type"] = "way" 
    
    # main attributes
    for key in elem.attrib:
        
        # grapping the main attributes
         ##############################################
        # main node info
        if key in PRIMARY:
            way[key] = elem.attrib.get(key) 
        
        ##############################################
        # sub node info
        elif key in CREATION:
            creation[key] = elem.attrib.get(key) 
                
        elif key in GEOPOS:
            geopos[key] = elem.attrib.get(key)
    
    # grapping sub info
    for tag in elem.iter():
        if tag.tag == "tag":
            
            ##############################################
            # main node info
            if tag.attrib['k'] in W_PRIMARY_TAG:
                 way[tag.attrib['k']] = tag.attrib['v']
            
            ##############################################
            # sub node info
            ##### all atts for creation dict
            elif tag.attrib['k'] in CREATION:
                creation[tag.attrib['k']] = tag.attrib['v']  
                
            ##### all atts for address dict
            elif ADDRESS in tag.attrib['k']:
                 # cleaning street data
                if is_street_name(tag):
                    address[tag.attrib['k'].replace("addr:", "")] = update_name(tag.attrib['v'], street_name_mapping)
                # cleaning city data
                elif is_city(tag):
                    address[tag.attrib['k'].replace("addr:", "")] = correct_city_spelling(remove_state_abbr(tag.attrib['v']))
                # all other cases
                else:
                    address[tag.attrib['k'].replace("addr:", "")] = tag.attrib['v']  
                
            ##### all atts for additional information dict
            elif (tag.attrib['k'] in W_ADDINFO) or (tag.attrib['k'] in W_OBJECT_TAG):
                
                # special cases if way has a amenity tag
                if is_amenity(tag):
                    addinfo["type"] = remove_chars(tag.attrib['v'])
                
                # other types of shops
                elif is_shop(tag):
                    try:
                        if len(addinfo["type"]) == 0:
                            pass
                        else:
                            addinfo["type_details"] = "shop"
                            addinfo["shop_type"] = remove_chars(tag.attrib['v'])
                    except:
                        addinfo["type"] = "shop"
                        addinfo["shop_type"] = remove_chars(tag.attrib['v'])
                    
                # other types of shops
                elif tag.attrib['k'] in ["highway", "leisure", "natural"]:
                    addinfo["type"] = tag.attrib['k']
                    addinfo["type_details"] = tag.attrib['v']
                    
                elif is_building(tag) and tag.attrib['v'] != "yes":
                    building["building_type"] = tag.attrib['v']
                    try:
                        if len(addinfo["type"]) == 0:
                            addinfo["type"] = "building"
                        else:
                            pass
                    except:
                        addinfo["type"] = "building"
                
                elif is_building(tag) and tag.attrib['v'] == "yes":
                    try:
                        if len(addinfo["type"]) == 0:
                            addinfo["type"] = "building"
                        else:
                            pass
                    except:
                        addinfo["type"] = "building"
                
                else:
                    addinfo[tag.attrib['k']] = tag.attrib['v']  
            
            ##### all atts for GNIS information dict
            elif CONTACT in tag.attrib['k']:
                contact[tag.attrib['k'].replace("contact:", "")] = tag.attrib['v']  
                
            ##### all atts for GNIS information dict
            elif BUILDING in tag.attrib['k'] or \
                 tag.attrib['k'] in ["name:historic", "architect", "building"]:
                # capture all attributes in a sub sub node
                building[tag.attrib['k'].replace("building:", "")] = tag.attrib['v']
                try:
                    if len(addinfo["type"]) == 0:
                        addinfo["type"] = "building"
                    else:
                        pass
                except:
                    addinfo["type"] = "building"
                
            ##### all atts for GNIS information dict
            elif GNIS in tag.attrib['k'] or tag.attrib['k'] == 'ele':
                gnis[tag.attrib['k'].replace("gnis:", "")] = tag.attrib['v']  
                
            ##### all atts for TIGER information dict
            elif TIGER in tag.attrib['k']:
                tiger[tag.attrib['k'].replace("tiger:", "")] = tag.attrib['v']  
            
            ##### all atts for other dict
            else:
                other[tag.attrib['k']] = tag.attrib['v']   
        
        elif tag.tag == "nd":
            nd = {}
            nd["ref"]  = tag.attrib['ref']
            nds.append(nd)            
    
    # put all the pieces together as they are available
    if len(creation) > 0:
        way["creation"] = creation
    if len(address) > 0:
        way["address"]  = address
    if len(building) > 0:
        addinfo["building"] = building
    if len(area) > 0:
        way["area"]     = area
    if len(geopos) > 0:
        way["geopos"]   = geopos
    if len(addinfo) > 0:
        way["addinfo"]  = addinfo
    if len(gnis) > 0:
        way["gnis"]     = gnis
    if len(tiger) > 0:
        way["tiger"]    = tiger
    if len(nds) > 0:
        way["nds"]    = nds
    if len(other) > 0:
        way["other"]    = other
    
    # Need to free up variables
    # in order to get RAM utilization managed
    creation = {} 
    address  = {}  
    contact  = {}  
    building = {}  
    area     = {}  
    geopos   = {}  
    addinfo  = {}  
    gnis     = {}  
    tiger    = {}  
    nds      = []  
    other    = {}  

    
    # return the node as a dict object
    return way

In [56]:
# convert a relation object to a python dict / JSON object string
# INPUT: elem - element in XML data
# RETURN: node - in JSON string format
def convert_relation_to_json(elem):
     # NODE dict
    relation = {} # main information container
    
    # SUB dicts
    creation = {}  # basic info about data creation
    address  = {}  # geopraphic information
    contact  = {}  # contact information
    building = {}  # building information
    area     = {}  # area information
    geopos   = {}  # geopraphic information
    addinfo  = {}  # some additional info about the datapoint
    gnis     = {}  # all GNIS attributes for the datapoint
    tiger    = {}  # all TIGER attributes for the datapoint
    members  = []  # all members of datapoint
    other    = {}  # all other info
    
    # type is always "node" 
    relation["type"] = "relation" 
    
    # main attributes
    for key in elem.attrib:
        
        # grapping the main attributes
         ##############################################
        # main node info
        if key in PRIMARY:
            relation[key] = elem.attrib.get(key) 
        
        ##############################################
        # sub node info
        elif key in CREATION:
            creation[key] = elem.attrib.get(key) 
    
    # grapping sub info
    for tag in elem.iter():
        if tag.tag == "tag":
            
            ##############################################
            # main node info
            if tag.attrib['k'] in R_PRIMARY_TAG:
                 relation[tag.attrib['k']] = tag.attrib['v']
            
            ##############################################
            # sub node info
            ##### all atts for creation dict
            elif tag.attrib['k'] in CREATION:
                creation[tag.attrib['k']] = tag.attrib['v']  
                
            ##### all atts for address dict
            elif ADDRESS in tag.attrib['k']:
                 # cleaning street data
                if is_street_name(tag):
                    address[tag.attrib['k'].replace("addr:", "")] = update_name(tag.attrib['v'], street_name_mapping)
                # cleaning city data
                elif is_city(tag):
                    address[tag.attrib['k'].replace("addr:", "")] = correct_city_spelling(remove_state_abbr(tag.attrib['v']))
                # all other cases
                else:
                    address[tag.attrib['k'].replace("addr:", "")] = tag.attrib['v']  
                
            ##### all atts for additional information dict
            elif (tag.attrib['k'] in R_ADDINFO) or (tag.attrib['k'] in R_OBJECT_TAG):
                
                # special cases if way has a amenity tag
                if is_amenity(tag):
                    addinfo["type"] = remove_chars(tag.attrib['v'])
                
                # other types of shops
                elif is_shop(tag):
                    try:
                        if len(addinfo["type"]) == 0:
                            pass
                        else:
                            addinfo["type_details"] = "shop"
                            addinfo["shop_type"] = remove_chars(tag.attrib['v'])
                    except:
                        addinfo["type"] = "shop"
                        addinfo["shop_type"] = remove_chars(tag.attrib['v'])
                    
                # other objct types
                elif tag.attrib['k'] in ["highway", "leisure", "natural"]:
                    addinfo["type"] = tag.attrib['k']
                    addinfo["type_details"] = tag.attrib['v']
                    
                elif is_building(tag) and tag.attrib['v'] != "yes":
                    building["building_type"] = tag.attrib['v']
                    try:
                        if len(addinfo["type"]) == 0:
                            addinfo["type"] = "building"
                        else:
                            pass
                    except:
                        addinfo["type"] = "building"
                
                elif is_building(tag) and tag.attrib['v'] == "yes":
                    try:
                        if len(addinfo["type"]) == 0:
                            addinfo["type"] = "building"
                        else:
                            pass
                    except:
                        addinfo["type"] = "building"
                
                else:
                    addinfo[tag.attrib['k']] = tag.attrib['v']  
            
            ##### all atts for GNIS information dict
            elif CONTACT in tag.attrib['k']:
                contact[tag.attrib['k'].replace("contact:", "")] = tag.attrib['v']  
                
            ##### all atts for GNIS information dict
            elif BUILDING in tag.attrib['k'] or \
                 tag.attrib['k'] in ["name:historic", "architect", "building"]:
                # capture all attributes in a sub sub node
                building[tag.attrib['k'].replace("building:", "")] = tag.attrib['v']
                try:
                    if len(addinfo["type"]) == 0:
                        addinfo["type"] = "building"
                    else:
                        pass
                except:
                    addinfo["type"] = "building"
                    
            ##### all atts for GNIS information dict
            elif GNIS in tag.attrib['k'] or tag.attrib['k'] == 'ele':
                gnis[tag.attrib['k'].replace("gnis:", "")] = tag.attrib['v']  
                
            ##### all atts for TIGER information dict
            elif TIGER in tag.attrib['k']:
                tiger[tag.attrib['k'].replace("tiger:", "")] = tag.attrib['v']  
            
            ##### all atts for other dict
            else:
                if is_type(tag):
                    relation["relation_type"] = tag.attrib['v']
                else:
                    other[tag.attrib['k']] = tag.attrib['v']   
             
        elif tag.tag == "member":
            member = {}
            member["type"] = tag.attrib['type']
            member["ref"]  = tag.attrib['ref']
            member["role"] = tag.attrib['role']
            members.append(member)
    
    # put all the pieces together as they are available
    if len(creation) > 0:
        relation["creation"] = creation
    if len(address) > 0:
        relation["address"]  = address
    if len(building) > 0:
        addinfo["building"] = building
    if len(area) > 0:
        relation["area"]     = area
    if len(addinfo) > 0:
        relation["addinfo"]  = addinfo
    if len(gnis) > 0:
        relation["gnis"]     = gnis
    if len(tiger) > 0:
        relation["tiger"]    = tiger
    if len(members) > 0:
        relation["members"]    = members
    if len(other) > 0:
        relation["other"]    = other
    
    # Need to free up variables
    # in order to get RAM utilization managed
    creation = {}
    address  = {}
    contact  = {}
    building = {}
    area     = {}
    geopos   = {}
    addinfo  = {}
    gnis     = {}
    tiger    = {}
    members  = []
    other    = {}
    
    # return the node as a dict object
    return relation

In [39]:
# main data cleaning and concersion funtion
# INPUT: osmfile - filename of XML file
# RETURN: data_final - array with JSON data inside
def convert_to_json(osmfile):
    
    # open xml file
    osm_file = open(osmfile, "r")
    
    # main data containers
    data_final = []
    
    # line by line walk
    for event, elem in ET.iterparse(osm_file):
        
        # main data nodes
        osm = {}
        note = {}
        meta = {}
        bounds = {}
        node = {}
        way = {}
        relation = {}
        
        # sub data nodes
        tags_cont = []
        nds_cont = []
        members_cont = []
        
        ##############################################
        # converting header information nodes
        ##############################################
        if elem.tag == "osm":
            osm["type"] = "osm"
            osm["content"] = elem.attrib
            if len(osm) > 0:
                data_final.append(osm)
        
        elif elem.tag == "note":
            note["type"] = "note"
            note["content"]= elem.text
            if len(note) > 0:
                data_final.append(note)
        
        elif elem.tag == "meta":
            meta["type"] = "meta"
            meta["content"] = elem.attrib
            if len(meta) > 0:
                data_final.append(meta)
        
        elif elem.tag == "bounds":
            bounds["type"] = "bounds"
            bounds["content"] = elem.attrib
            if len(bounds) > 0:
                data_final.append(bounds)
        
        ##############################################
        # converting node tags
        ##############################################
        elif elem.tag == "node":
            # call function to convert node data into json
            node = convert_node_to_json(elem)
            # add to nodes
            if len(node) > 0:
                data_final.append(node)
        elif elem.tag == "way":
            # call function to convert node data into json
            way = convert_way_to_json(elem)
            # add to nodes
            if len(way) > 0:
                data_final.append(way)
        elif elem.tag == "relation":
            # call function to convert node data into json
            relation = convert_relation_to_json(elem)
            # add to nodes
            if len(relation) > 0:
                data_final.append(relation)    
                
    # Need to free up variables
    # in order to get RAM utilization managed                
    # main data nodes
    osm = {}
    note = {}
    meta = {}
    bounds = {}
    node = {}
    way = {}
    relation = {}
    tags_cont = []
    nds_cont = []
    members_cont = []     
                
        
    return data_final

In [201]:
# convert the XML file into the defined JSON format
# call of main XML to JSON convertion
data = convert_to_json(filename)

In [202]:
# definition of json file name
json_filename = "waukesha_county_osm.json"

# create json file based on python dict prepared above
with open(json_filename, 'w') as outfile:
    json.dump(data, outfile)

# free up variable
data = None

In [203]:
# Return the size, in bytes, of path. Raise os.error if the file does not exist or is inaccessible.
print "Size of OSM JSON data file: "
print (str(os.path.getsize(json_filename)/1024)) + " KB"

Size of OSM JSON data file: 
243607 KB


### Store data in MongoDB

In [204]:
# get client to local mongoDB instance
client = pymongo.MongoClient("mongodb://localhost:27017")

db_name = "osm_data"

# try to create new database osm_data, drop and recreate if already exists
try:
    client.drop_database(db_name)
    db = client.osm_data
    print "Successfully dropped existing DB and created database " + str(db_name)

# catch case if database does not exist and therfore can not be dropped
except:
    try:
        db = client.osm_data
        print "Successfully created new database "  + str(db_name)
    except:
        print "Issues creating new Database " + str(db_name)

Successfully dropped existing DB and created database osm_data


In [205]:
with open(json_filename) as f:
    data_for_import = json.loads(f.read())
    # perform insert to MongoDB
    db.wau_county.insert_many(data_for_import)

# free up variable    
data_for_import = None

#### Creating some indexes which will speed up our queries later on

In [206]:
db.wau_county.create_index("id")
db.wau_county.create_index("type")
db.wau_county.create_index("addinfo.type")
db.wau_county.create_index("creation.user")
db.wau_county.create_index("address.city")
print "Indexes created"

Indexes created


### Query MongoDB

In [63]:
# Number of Documents
num_docs  = db.wau_county.find().count()
num_nodes = db.wau_county.find({"type":"node"}).count()
num_ways  = db.wau_county.find({"type":"way"}).count()
num_rels  = db.wau_county.find({"type":"relation"}).count()

print('--------------- Key Types & Count ------------------')
print('Nr. of Documents: ' + str(num_docs))
print('--------------------------')
print('Nr. of Nodes:     ' + str(num_nodes))
print('Nr. of Ways:      ' + str(num_ways))
print('Nr. of Relations: ' + str(num_rels))

--------------- Key Types & Count ------------------
Nr. of Documents: 953864
--------------------------
Nr. of Nodes:     860835
Nr. of Ways:      92239
Nr. of Relations: 786


The numbers for Nodes, Ways and Relations is consistent with the numbers gathered earlier on XML basis (see Key Types & Count). Means, thus far it looks like the migration from XML to JSON as well as the import into MongoDB was successful.

In [64]:
print('--------------- Users ------------------')
unique_users = len(db.wau_county.distinct("creation.user"))
print('Number of unique users: ' + str(unique_users))
print('----------------------------------------')
aggregation_query = [{"$group" : {"_id" : "$creation.user", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}},
                     {"$limit" : 10}]
top_users = db.wau_county.aggregate(aggregation_query)
print('First 10 users in the list:')
for user in top_users:
    print user["_id"] + ": " + str(user["count"])
print('----------------------------------------')

--------------- Users ------------------
Number of unique users: 682
----------------------------------------
First 10 users in the list:
woodpeck_fixbot: 217089
ItalianMustache: 102449
shuui: 94917
hogrod: 48592
reschultzed: 40931
bbauter: 29704
Mulad: 28162
Gary Cox: 27193
iandees: 26923
TIGERcnl: 25893
----------------------------------------


The top 10 users and the number of their posts is consistent with the numbers gathered earlier on XML basis (see Users). Means again, thus far it looks like the migration from XML to JSON as well as the import into MongoDB was successful.

In [196]:
print '------------------ Amenity Types -------------------'
print 'Below is a list of different amenities and and their'
print 'occurence in the data.'
print '----------------------------------------------------'
aggregation_query = [
                     {"$match" : { "addinfo.type" : {"$exists" : True } }},
                     {"$group" : {"_id" : "$addinfo.type", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}},
                     {"$limit" : 10}
                    ]
info_types = db.wau_county.aggregate(aggregation_query)
print('First 10 users in the list:')
for info in info_types:
    print str(info["_id"]) + ": " + str(info["count"])
print('----------------------------------------')

------------------ Amenity Types -------------------
Below is a list of different amenities and and their
occurence in the data.
----------------------------------------------------
First 10 users in the list:
highway: 62601
building: 11053
leisure: 2438
parking: 2304
natural: 2012
school: 997
shop: 694
parking entrance: 569
restaurant: 444
fast food: 230
----------------------------------------


Amenities list looks a bit different now. Reason for that is that the data has been restructured and thereby amenity information, but also other information like highway, building, leisure, natural, etcs. has been clubbed together into a new addinfo.type attribute. In that attribute basically describes what the data point is, independent of the main type (node, relation, way). It provides a more comprehensive insight to the data and makes searching for specific types of nodes, ways and relations easier.

However, there was an issue I had to invest in detail. Sometime, the count in the XML data was higher than the one in the MongoDB. An example for that is: school, restaurant, shops. During my investigations I found out that sometimes the migration procedure from XML to JSON was overwriting some values for "addinfo.type" in JSON format. For example if there was a restaurant which was a butcher shop too, in addition of being a restaurant. 

One example of an issue with an "amenity" = "parking" which was migrated to an "addinfo.type" = "leisure" is added in the submission in file issue_example.txt. Here, due to the restructuring of some data during the migration process, the value for "addinfo.type" is overwritten by "addinfo.type" = "leisure".

In [83]:
print '------------------ Post Codes -------------------'
print 'Below is a list of all post codes and and their'
print 'occurence in the data.'
print '-------------------------------------------------'
aggregation_query = [
                     {"$match" : { "address.postcode" : {"$exists" : True }}},
                     {"$group" : {"_id" : "$address.postcode", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}},
                     {"$limit" : 10}
                    ]
post_codes = db.wau_county.aggregate(aggregation_query)
print('First 10 post codes in the list:')
for post_code in post_codes:
    print str(post_code["_id"]) + ": " + str(post_code["count"])
print('----------------------------------------')

------------------ Post Codes -------------------
Below is a list of all post codes and and their
occurence in the data.
-------------------------------------------------
First 10 post codes in the list:
53202: 993
53212: 255
53203: 229
53538: 143
53233: 132
53211: 93
53204: 67
53215: 47
53094: 42
53027: 38
----------------------------------------


In [89]:
print '-------------- Correct Post Code Types ---------------'
print 'Below is a list of post codes which do not belong to'
print 'Waukesha County and their occurence in the data.'
print '----------------------------------------------------'
aggregation_query = [
                     {"$match" : { "address.postcode" : {"$exists" : True } } },
                     {"$group" : {"_id" : "$address.postcode", "count" : {"$sum" : 1} } },
                     {"$sort" : {"count":-1} }
                    ]
post_codes = db.wau_county.aggregate(aggregation_query)
print('First 10 post codes in the list:')
for post_code in post_codes:
    # this checks if the post code is in the post codes of waukesha county area
    if post_code["_id"] in post_code_expected:
        print str(post_code["_id"]) + ": " + str(post_code["count"])
print('----------------------------------------')

-------------- Correct Post Code Types ---------------
Below is a list of post codes which do not belong to
Waukesha County and their occurence in the data.
----------------------------------------------------
First 10 post codes in the list:
53186: 23
53029: 23
53066: 12
53072: 11
53045: 11
53051: 10
53005: 9
53188: 8
53151: 7
53069: 3
53146: 3
53149: 3
53189: 2
53122: 2
53089: 2
53150: 2
53183: 1
53018: 1
53103: 1
53007: 1
53187: 1
53153: 1
----------------------------------------


In [112]:
print '-------------- Wrong Post Code Types ---------------'
print 'Below is a list of post codes which do not belong to'
print 'Waukesha County and their occurence in the data.'
print '----------------------------------------------------'
c = 0
aggregation_query = [
                     {"$match" : { "address.postcode" : {"$exists" : True }}},
                     {"$group" : {"_id" : "$address.postcode", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
post_codes = db.wau_county.aggregate(aggregation_query)
print('First 10 post codes in the list:')
for post_code in post_codes:
    c = c+1
    # this checks if the post code is in the post codes of waukesha county area
    if post_code["_id"] not in post_code_expected:
        print str(post_code["_id"]) + ": " + str(post_code["count"])
    
    # exit loop after 10 times  
    if c == 10:
        break
print('----------------------------------------')

-------------- Wrong Post Code Types ---------------
Below is a list of post codes which do not belong to
Waukesha County and their occurence in the data.
----------------------------------------------------
First 10 post codes in the list:
53202: 993
53212: 255
53203: 229
53538: 143
53233: 132
53211: 93
53204: 67
53215: 47
53094: 42
53027: 38
----------------------------------------


In [109]:
print '-------------- Wrong Post Code Types ---------------'
print 'Below is a list of post codes which do not match an'
print 'extended pattern around Waukesha County and their'
print 'occurence in the data.'
print 'Pattern: (1) 53000 - 53599 and'
print '         (2) length of post code is 5 chars'
c = 0
aggregation_query = [
                     {"$match" : { "address.postcode" : {"$exists" : True }}},
                     {"$group" : {"_id" : "$address.postcode", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
post_codes = db.wau_county.aggregate(aggregation_query)
for post_code_info in post_codes:
    post_code = post_code_info["_id"]
    # check agains regex for valid post codes
    is_valid_post_code = post_code_re.match(post_code)
    # if post code does not match expected pattern or post code length is not 5
    if (not is_valid_post_code) or (len(post_code) != 5):
        pprint.pprint(post_code)
    # exit loop after 10 times  
    if c == 10:
        break
print('----------------------------------------')

-------------- Wrong Post Code Types ---------------
Below is a list of post codes which do not match an
extended pattern around Waukesha County and their
occurence in the data.
Pattern: (1) 53000 - 53599 and
         (2) length of post code is 5 chars
u'54220'
u'1729'
u'53203-3099'
u'53217-5399'
u'53214-3110'
u'53403-9998'
u'Milwaukee WI, 53222'
u'WI'
----------------------------------------


In [110]:
print '-------------- Cities in the dataset ---------------'
print 'Below is a list of different cities mentioned in the'
print 'data and their occurence.'
print '----------------------------------------------------'
aggregation_query = [{"$match" : { "address.city" : {"$exists" : True }}},
                     {"$group" : {"_id" : "$address.city", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}},
                     {"$limit" : 10}
                    ]
cities = db.wau_county.aggregate(aggregation_query)
print('First 10 users in the list:')
for city in cities:
    print str(city["_id"]) + ": " + str(city["count"])
print('----------------------------------------')

-------------- Cities in the dataset ---------------
Below is a list of different cities mentioned in the
data and their occurence.
----------------------------------------------------
First 10 users in the list:
Milwaukee: 1622
Racine: 561
Fort Atkinson: 145
Mount Pleasant: 120
Waterloo: 57
Watertown: 46
Waukesha: 34
Caledonia: 29
Sturtevant: 27
Hartland: 21
----------------------------------------


The counts for the different cities in the list changed between earlier data analysis on the XML data and now. Reason for the different numbers is the data cleaning done when transferring the data from XML to JSON. 

Example: Waukesha 
- Before data cleaning: 
        - Waukesa: 1
        - Waukesha: 31
        - Waukesha, WI: 1
        - waukesha: 1
- After data cleaning:
        - Waukesha: 34
        
However, what's strange when looking at the different cities in the data, is that the cities with the highest occurences in the data, eg. Milwaukee, do not even below to Waukesha county. To be sure that is not an issue of the investigations on my side, I doulbechecked all everything from the beginning again... selection on OpenStreetMap, list of post codes, and so on.

In [229]:
aggregation_query = [{"$match" : {"addinfo.religion" : {"$exists":1} , "addinfo.type" : "place of worship"}},
                     {"$group" : {"_id" : "$addinfo.religion", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
places_of_worship = db.wau_county.aggregate(aggregation_query)

print('-----------------------------------------')
print('---------- Places of Worship ------------')
print('- Information on the RELIGION of Places -')
print('-----------------------------------------')
for place in places_of_worship:
    print str(place["_id"]).title() + ": " + str(place["count"])
print('----------------------------------------')

-----------------------------------------
---------- Places of Worship ------------
- Information on the RELIGION of Places -
-----------------------------------------
Christian: 52
Buddhist: 2
Jain: 1
Sikh: 1
Jewish: 1
Muslim: 1
Scientologist: 1
Hindu: 1
----------------------------------------


Above information is displaying all the different religions places which are mentioned in the data. By far, Christian's places are most frequent with 52 appearances in the data. In the next section we take a closer look at those Christian's places and what denomination they follow.

In [232]:
aggregation_query = [{"$match" : {"addinfo.religion" : "christian" , "addinfo.type" : "place of worship"}},
                     {"$group" : {"_id" : "$addinfo.denomination", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
places_of_worship = db.wau_county.aggregate(aggregation_query)

print('---------------------------------------------------')
print('--------------- Places of Worship -----------------')
print('- Information on the DENOMINATION of Christian\'s -')
print('---------------------------------------------------')
for place in places_of_worship:
    print str(place["_id"]).title() + ": " + str(place["count"])
print('---------------------------------------------------')

---------------------------------------------------
--------------- Places of Worship -----------------
- Information on the DENOMINATION of Christian's -
---------------------------------------------------
None: 19
Lutheran: 13
Catholic: 8
Episcopal: 2
Methodist: 1
Mormon: 1
Evangelical: 1
Presbyterian: 1
Nondenominational: 1
Congregational: 1
Wisconsin_Evangelical_Lutheran_Synod_(Wels): 1
Baptist: 1
Pentecostal: 1
Greek_Orthodox: 1
---------------------------------------------------


For a good portion of all Christian places, there is no information about the denomination provided. However, from the data we have, we can see that Lutheran and Catholic denomination are most popular. _Note: What we can see here as well is a good of example of bad data quality. One denomination records has a value "Wisconsin_Evangelical\_Lutheran\_Synod\_(Wels)". That seems to be wrong._

### Other Ideas about the Data set

In this section I will do some more investigations on the data set and point out chances for improvement of the data as well as some more interesting facts.

In [144]:
aggregation_query = [{"$match" : {"addinfo.type" : {"$exists":1} , "addinfo.type" : "restaurant"}},
                     {"$group" : {"_id" : "$addinfo.cuisine", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
restaurants = db.wau_county.aggregate(aggregation_query)

c=0
print('----------------------------------------')
print('-------- Cuisine in Restaurants --------')
print('----------------------------------------')
for restaurant in restaurants:
    print str(restaurant["_id"]).title() + ": " + str(restaurant["count"])
    c = c+restaurant["count"]
print('----------------------------------------')

print "Number of Restaurants in the data: " + str(c)

----------------------------------------
-------- Cuisine in Restaurants --------
----------------------------------------
None: 241
American: 28
Pizza: 27
Sandwich: 16
Italian: 16
Chinese: 12
Burger: 11
Japanese: 7
Mexican: 7
American_New: 7
Regional: 4
Greek: 3
German: 3
Steak_House: 3
New_Orleans: 2
New_American: 2
Bagel: 2
Irish: 2
Sandwich: 2
Asian: 2
Indian: 2
Korean: 2
Seafood: 2
Diner: 2
Vietnamese: 2
Ethiopian: 2
Sushi: 2
Chili: 1
New_American; European: 1
Wrap: 1
American;Mediterranean: 1
Chinese: 1
European: 1
Fast_Food: 1
Noodles: 1
Everything: 1
Brunch: 1
Vietnamese,Asian: 1
Ice_Cream: 1
Irish: 1
International: 1
Sub: 1
Turkish: 1
Asian_Fusion: 1
Deli: 1
World: 1
Cuban: 1
American_New: 1
Pancake: 1
Thai: 1
Burger,_Pizza: 1
Gastropub: 1
Sandwich; Thai: 1
Seafood; Grill: 1
Pizza,_Burgers,_And_Fish_Frys: 1
Steak: 1
Dutch: 1
Bbq: 1
Pasta: 1
Brazilian: 1
----------------------------------------
Number of Restaurants in the data: 444


__What needs to be improved?__
What we can see here is that we got a good number of 372 restaurants on the map for Waukesha County area already. However, where we can see room for improvement is definitley the specification of the cuisine. Only a about 130 restaurants provide cuisine information, where about 240 do not do so. 


__How to improve?__
What's easy, is to find out which restaurant do no have cusine information provided and to additionally add certain information like website, phone, name, operator to that report, to enable potential "data cleaners" to request the data from a certain source. Where the report (below a sample report is provided, it can be improved to further support the "data cleaner") itself is a simple job which can be done by a data analyst, the retrieving and cleaning of the data requires manual effort and therefore might be quite a challange. _One the sample Report below: Already the name of most restaurants tells a lot about what cuisines is offererd, at least about a portion of it. Eg. Cold Spoons Gelato, Cranky Al's Bakery and Pizza, Crisp Pizza Bar and Lounge, Depot Snack Shop, Chiang Mai Thai, Chipotle Bar and Grill, Pitch's BBQ,..._


__What are the benefits?__
As especially restaurants might be locations which are interesting targets for map users, improved information would help increadying user satisfaction of OpenStreetMap.

In [189]:
restaurants_wo_cusine = db.wau_county.find({"addinfo.type"    :"restaurant", 
                                            "addinfo.cuisine" : {"$exists":0}
                                           })

all_attrs = {"id" : "ID",
        "name" : "Name",
        "operator" : "Operator",
        "phone" : "Phone",
        "website" : "Website",
        "opening_hours" : "Opening Hours"
       }

# as this is only a sample report, the output lines are limited to 15
c = 0
c_max = 15

print "---------------------------------------------------"
print "---- SAMPLE REPORT FOR FILLING RESTAURANT INFO ----"
print "---------------------------------------------------"

for restaurant in restaurants_wo_cusine:
    
    # counter for sample report
    c = c+1
    if c > c_max:
        break
    
    # write results
    for attr in all_attrs.keys():
        try:
            print all_attrs[attr] + ": " + restaurant[attr]
        except:
            pass
    print '--------------------------------------------'


---------------------------------------------------
---- SAMPLE REPORT FOR FILLING RESTAURANT INFO ----
---------------------------------------------------
Name: Robert's Frozen Custard
ID: 471082814
--------------------------------------------
Name: Meritage
ID: 475437538
--------------------------------------------
Name: Cold Spoons Gelato
ID: 475442109
--------------------------------------------
Name: Cranky Al's Bakery and Pizza
ID: 475446467
--------------------------------------------
Name: Fazoli's
ID: 545357522
--------------------------------------------
Name: Perkins
ID: 545367151
--------------------------------------------
Name: Panera
Opening Hours: Mo-Sa 06:00-20:30; Su 07:00-19:00
ID: 573374681
--------------------------------------------
Name: Café Lulu
ID: 573538503
--------------------------------------------
Name: Benji's Deli
ID: 573736642
--------------------------------------------
Name: Denny's
ID: 585911973
--------------------------------------------
Name: Com

In [169]:
aggregation_query = [{"$match" : {"addinfo.type" : {"$exists":1}, "type" : "way" } },
                     {"$group" : {"_id" :"$addinfo.type", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
ways = db.wau_county.aggregate(aggregation_query)

print('----------------------------------------')
print('------- Different Types of Ways --------')
print('----------------------------------------')
for way in ways:
    print str(way["_id"]).title() + ": " + str(way["count"])
print('----------------------------------------')

----------------------------------------
------- Different Types of Ways --------
----------------------------------------
Highway: 62593
Building: 11027
Leisure: 2392
Parking: 2177
Natural: 1938
School: 328
Restaurant: 129
Place Of Worship: 103
Fast Food: 72
Fuel: 55
Shelter: 52
Bank: 48
Bar: 37
Theatre: 26
Toilets: 24
Fire Station: 21
Grave Yard: 20
Hospital: 19
Pharmacy: 18
Cafe: 17
Library: 16
Clinic: 13
Community Centre: 12
Townhall: 12
Post Office: 12
Car Wash: 10
College: 10
Pub: 10
Bicycle Rental: 10
University: 8
Social Facility: 7
Dentist: 6
Nightclub: 6
Police: 6
Events Venue: 6
Bicycle Parking: 6
Public Building: 6
Cinema: 5
Marketplace: 5
Social Centre: 4
Courthouse: 4
Recycling: 4
Prison: 4
Boatyard: 3
Motorcycle Parking: 3
Dojo: 3
Biergarten: 3
Swimming Pool: 3
Nursing Home: 3
Car Sharing: 2
Fountain: 2
Ranger Station: 2
Ice Cream: 2
Studio: 2
Healthcare: 2
Veterinary: 2
Parking Space: 2
Boat Storage: 1
Car Rental: 1
Architect Office: 1
Hospice: 1
Arts Centre: 1
Mortuary

__What needs to be improved?__

Above you can see a list of different types of ways inside the dataset. Obviously, ways are not only streets, roads, avenues, paths,... All different kind of amenities have entered the way tags information. Altough, as per the definition of Way objects in the OpenStreetMap Wiki, Way objects are not designed to store such data. See link to OpenStreetMap Wiki below.

__How to improve?__

Programmatic ways seem to be applicable to clean the data to a certain degree. By combining different pieces of information (the Way object information itself, related/referenced objects) required information to build up Node objects based on that information should be availalbe. Although, it's required to develop a broader conceptional design to fully understand the scope of such a conversion project. Quickly drafting my thoughts: It looks like lot's of information within <tag>'s can be used to build up the Node objects info like "creation", "maininfo", "addinfo" (based on how I classify objects in JSON), from the referenced objects it should be possible to gather information about "geopos".


In [ ]:
# EXAMPLE OF WAY ELEMENT WHICH 
<way id="381049573" version="1" timestamp="2015-11-18T18:31:24Z" changeset="35414977" uid="1952296" user="shuui">
    <nd ref="1008172080"/> # SEE EXAMPLE BELOW FOR REFERENCE
    <nd ref="3843184249"/>
    <nd ref="3843184250"/>
    <nd ref="1008172388"/>
    <nd ref="1008171880"/>
    <nd ref="3843184251"/>
    <nd ref="1008172049"/>
    <nd ref="1008172080"/> # SEE EXAMPLE BELOW FOR REFERENCE
    <tag k="addr:city" v="Milwaukee"/>
    <tag k="addr:housenumber" v="209"/>
    <tag k="addr:postcode" v="53204"/>
    <tag k="addr:state" v="WI"/>
    <tag k="addr:street" v="South Water Street"/>
    <tag k="amenity" v="architect_office"/>
    <tag k="building" v="commercial"/>
    <tag k="building:levels" v="1"/>
    <tag k="building:year_built" v="2015"/>
    <tag k="name" v="pra Plunkett Raysich Archetects, LLP"/>
    <tag k="phone" v="+1.800.208.7078"/>
    <tag k="website" v="http://prarch.com"/>
  </way>

# EXAMPLE OF NODE REFERENCES BY WAY OBJECT
<node id="1008172080" lat="43.0292647" lon="-87.9083026" version="3" timestamp="2015-11-18T18:31:24Z" changeset="35414977" uid="1952296" user="shuui"/>

__What are the benefits?__

Cleaning up the way tags information might support to get to more standardized data and thereby makes it easier to search for data, describe/explain certain data objects, build reports on the data, build navigation/view layers based on the data, and so on.


_Definition of a "Way" in OpenStreetMap:_ https://wiki.openstreetmap.org/wiki/Way

In [158]:
aggregation_query = [{"$match" : {"addinfo.type" : {"$exists":1}, "type" : "relation" } },
                     {"$group" : {"_id" :"$addinfo.type", "count" : {"$sum" : 1}}},
                     {"$sort" : {"count":-1}}
                    ]
ways = db.wau_county.aggregate(aggregation_query)

print('----------------------------------------')
print('------- Different Types of Ways --------')
print('----------------------------------------')
for way in ways:
    print str(way["_id"]).title() + ": " + str(way["count"])
print('----------------------------------------')

----------------------------------------
------- Different Types of Ways --------
----------------------------------------
Natural: 74
Leisure: 46
Building: 26
Parking: 9
Highway: 8
University: 3
School: 3
Library: 2
College: 2
Community Centre: 2
Restaurant: 2
Place Of Worship: 1
Fuel: 1
----------------------------------------


In [ ]:
http://wiki.openstreetmap.org/wiki/Relation